# Combination of all important functions
This is a summary of the important functions for constructing a network and solving it for the currents.   
First off there is the Construct function which takes the paths to the input data and builds a corresponding Networkx-Graph from it.  
Then there ist the Solve class which takes a network as input and can then construct the matrices and vectors needed to solve for the currents.   
In the end there is a snippet attached which can be used to orient the edges/current-directions. It takes a incidence-like matrix as input and changes the signs according to the potential landscape.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy import sparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def Construct(Coo, Typ, Ener):
    #Read coordinates from file
    coords = list(map(tuple, np.genfromtxt(Coo, delimiter=" ")))
    #Build Graph
    F=nx.Graph()
    F.add_nodes_from(coords)
    #Read Attributes
    types = np.genfromtxt(Typ, delimiter=" ")
    energies = np.genfromtxt(Ener, delimiter=" ")
    #Set Attributes
    i=0
    for u in F.nodes():
        F.nodes[u]["pos"]=u
        F.nodes[u]["type"]=types[i]
        F.nodes[u]["potential"]=energies[i]
        i+=1  
    #Construct Edges
    F.add_edges_from(nx.geometric_edges(F, 1))
    
    ## Inizializing with random values until access to resistivity values ##
    R_0=1
    sigmaR=0.05
    for (u, v) in F.edges(): #die beiden Endpunkte u und v der Kante
        F.edges[u,v]['weight'] = random.normalvariate(R_0, sigmaR)
        
    ## These Edges should maybe be oriented by using some variation of the code in the end ##
    
    return F


#Gitter=Construct("coord_0.dat", "mol_types_0.dat", "site_energies_0.dat")

In [3]:
class Solve:
    def __init__(self, X):
        self.G=X
        self.N=self.G.order()
        self.nR=self.G.number_of_edges()
        
    def resistor(self):
        #Extract the values of the resistors from the graph and build a nR x nR matrix
        mat=sparse.spdiags(list(nx.get_edge_attributes(self.G, "weight").values()), 0, self.nR, self.nR)
        return sparse.csc_matrix(mat)
    
    def incidence(self):
        #Builds the incidence matrix from the graph
        mat= np.transpose(nx.incidence_matrix(self.G, oriented=1)) #Beachte Transpose damit die Dimensionen der networkx funktion zum paper passen
        
        
        ## Siehe Codeschnipsel im ersten Beispiel falls die Orientierung ungünstig ist ##
        
        
        return mat
    
    def voltages(self):
        #Get the potential values from the nodes and build a vector
        vec=np.array(list(nx.get_node_attributes(self.G, "potential").values()))
        return vec

    
    def currents(self):
        #Combines the other functions to get the currents trough the resistors
        return - (sparse.linalg.inv(self.resistor()) @ self.incidence()) @ self.voltages()

    
#Solve(Gitter).currents()   

In [4]:
def Orientation(G):
    #Code Snippet that could be used for orienting the edges
    Mat=G.incidence()
    V=G.voltages()
    for x in range(np.shape(Mat)[0]):
        Y=[]
        for y in range(np.shape(Mat)[1]):
            if Mat[x,y] !=0:
                Y.append(y)
        Mat[x,Y[0]]=-np.sign(V[Y[0]]-V[Y[1]])
        Mat[x,Y[1]]=-Mat[x,Y[0]]
    return Mat

In [5]:
Gitter=Construct("coord_0.dat", "mol_types_0.dat", "site_energies_0.dat")
Setup=Solve(Gitter)
Setup.currents()

array([-0.04652197, -0.27476601,  0.22054761, -0.01420838,  0.12936561,
        0.06069344, -0.06843138, -0.1599633 , -0.04871175,  0.25057646,
       -0.17083087, -0.08658256,  0.12798222, -0.04635961, -0.05408911,
        0.03939641,  0.14994613, -0.21965199,  0.09155032, -0.06323699,
        0.12468943, -0.24102459,  0.04735155,  0.13223547, -0.1114012 ,
        0.23600958, -0.14935575,  0.0341291 , -0.00076017, -0.23537913,
        0.13824811, -0.08431975,  0.14343557, -0.07171559,  0.00452254,
        0.0101158 ,  0.20099836, -0.19238373,  0.11930384, -0.05442987,
        0.00360202,  0.12030944, -0.29076153,  0.19267266,  0.05255773,
       -0.09398163, -0.0192463 ])